In [4]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import arr_to_dict, grid_to_dict, neighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate, gridneigh
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [5]:
f=open('input.txt').read().split('\n')
lines = [line for line in f]
lines[0]


'122102002221220113213120331300324403204320144533424135412222404323003104143241322330203332311001201'

In [91]:
grid, neigh = gridneigh('test0.txt', parser=lambda x: [int(ch) for ch in x.split(',')])
grid, neigh = gridneigh('input.txt', to_dict = False, parser=lambda x: [int(ch) for ch in x])


In [92]:
rows = len(grid)
cols = len(grid[0])
rows, cols
scores = defaultdict(lambda x:1)

In [93]:
def look(r,c):
    overall = 1
    high = grid[r][c]
    score = 0
    for nr in range(r-1,-1,-1):
        if grid[nr][c] >= high:
            score +=1
            break
        else:
            score += 1
    # print(score)
    overall *= score
    score = 0
    for nr in range(r+1,rows):
        if grid[nr][c] >= high:
            score +=1
            break
        else:
            score += 1
    # print(score)
            
    overall *= score
    score = 0
    for nc in range(c-1,-1,-1):
        if grid[r][nc] >= high:
            score += 1
            break
        else:
            score += 1
    overall *= score
    # print(score)
    
    score = 0
    for nc in range(c+1,cols):
        if grid[r][nc] >= high:
            score +=1
            break
        else:
            score += 1
    # print(score)
            
    overall *= score
    return overall
ans = 0
for r in range(rows):
    for c in range(cols):
        ans = max(look(r,c), ans)
ans

315495

In [71]:
visible = set()
for r in range(rows):
    high = -1
    for c in range(cols):
        if grid[r][c] > high:
            high = grid[r][c]
            visible.add((r,c))

len(visible)

576

In [72]:
for r in range(rows-1,-1,-1):
    high = -1
    for c in range(cols-1,-1,-1):
        if grid[r][c] > high:
            high = grid[r][c]
            visible.add((r,c))

len(visible)

1137

In [73]:
for c in range(cols-1,-1,-1):
    high = -1
    for r in range(rows-1,-1,-1):
        if grid[r][c] > high:
            high = grid[r][c]
            visible.add((r,c))

len(visible)

1477

In [74]:
for c in range(cols-1,-1,-1):
    high = -1
    for r in range(rows):
        if grid[r][c] > high:
            high = grid[r][c]
            visible.add((r,c))

len(visible)

1812

In [53]:
for r,c in visible:
    if r!=0 and r!= 4 and c !=0 and c != 4:
        print(r,c, grid[r][c])

58 1 3
69 1 3
80 1 4
98 37 1
98 46 1
98 55 1
42 2 5
28 94 5
92 97 2
72 2 4
2 41 4
3 6 3
88 1 3
2 59 5
98 5 0
97 49 4
98 14 0
1 72 4
7 3 3
18 3 3
98 23 4
98 32 3
43 96 4
98 41 3
57 98 3
46 98 1
5 96 3
30 98 1
19 98 4
8 98 1
62 96 5
97 35 3
87 98 3
98 9 3
97 53 3
98 18 4
71 1 4
1 67 2
98 27 0
96 94 3
1 85 3
98 36 4
1 94 1
76 97 1
49 98 2
38 98 3
44 1 4
77 98 1
72 1 2
27 97 4
34 2 2
11 98 3
97 12 3
2 49 4
39 98 3
97 30 2
97 39 3
97 48 5
98 13 2
7 2 2
98 22 1
63 1 3
1 80 3
96 98 0
1 89 3
91 1 1
46 97 2
80 98 1
69 98 3
58 98 3
64 1 2
97 7 2
97 16 3
97 25 3
98 8 0
26 1 4
96 75 4
1 66 3
98 17 2
96 93 3
1 84 3
88 98 2
76 96 2
94 1 2
49 97 3
94 95 3
52 98 3
41 98 2
97 11 3
97 20 2
70 3 4
1 34 3
1 52 5
98 3 2
1 61 5
1 70 4
14 98 0
3 98 2
98 97 1
60 3 5
82 98 3
71 98 0
95 55 5
60 97 4
75 95 4
95 73 4
1 29 1
44 98 2
33 98 1
1 38 5
28 1 3
1 47 5
72 98 2
1 56 5
29 2 4
98 83 3
98 92 3
90 97 1
79 97 4
63 98 4
2 2 2
32 3 3
2 11 2
91 98 3
96 24 4
86 1 2
1 15 3
52 97 4
41 97 3
97 10 3
95 77 4
96 42 3
59 

In [ ]:
def graphparse(filename, parser=None):
    f = open(filename, 'r').read().splitlines()
    if not parser:
        conn = [(line.split()) for line in f]
    else:
        conn = [parser(line) for line in f]    
    return multidict(conn)
graphparse('input.txt', parser=lambda line: [x for x in line.split()])

ValueError: too many values to unpack (expected 2)